<div style="text-align: center;">

  <img src="img/LogoUesc.png" alt="UESC" style="width: 60px; display: block; margin: 0 auto 5px auto;">

  <div style="font-size: 20px;"><strong>UNIVERSIDADE ESTADUAL DE SANTA CRUZ - UESC</strong></div>
  <div style="font-size: 18px;">DEPARTAMENTO DE ENGENHARIAS E COMPUTAÇÃO - DEC</div>
  <div style="font-size: 18px;">Grupo de Pesquisas em Simulações e Controle de Processos</div>

  <hr style="margin: 20px 0; border: 1px solid #0074B7;">

  <div style="font-size: 22px; font-weight: bold; margin-bottom: 10px;">PROGRAMAÇÃO EM PYTHON PARA AS ENGENHARIAS</div>

  <div style="font-size: 18px; margin-top: 10px;"><strong>Tema</strong>: Introdução a Redes Neurais Artificiais - Básico</div>
  <div style="font-size: 18px;"><strong>Prof. Dr. E.R.Edwards</strong></div>

  <div style="font-size: 16px; margin-top: 15px;">Ilhéus - BA, Abril de 2025</div>
  
  <hr style="margin: 20px 0; border: 1px solid #0074B7;">

</div>

#### Exercício – Classificação do Tipo de Reator com Redes Neurais

__Enunciado__:

Em uma planta de processos químicos, diferentes reatores são utilizados conforme as características da reação, como a necessidade de agitação contínua, regime de operação ou tempo de residência. Os dados operacionais mais comuns utilizados no projeto e na escolha desses reatores incluem a __massa de catalisador__ e a __temperatura__ de operação.

A tabela a seguir apresenta informações de quatro reatores distintos. O objetivo é treinar uma rede neural que consiga classificar o tipo de reator com base nessas duas variáveis operacionais.

<img src="img/Tab_08.png" alt="UESC" style="width: 650px; display: block; margin: 0 auto 5px auto;">
<p style="text-align: center;"><strong>Figura: Tabela de dados para previsão de novas correntes.</strong></p>

__Objetivo__:

Desenvolver uma rede neural simples de classificação que receba como entrada a __massa de catalisador__ e a __temperatura__, e como saída o __tipo de reator__.

__Instruções__:

1. Codifique os tipos de reator com vetores binários (ex: __CSTR__ = $[1, 0, 0]$, __PFR__ = $[0, 1, 0]$, __BATCH__ = $[0, 0, 1]$)
2. Normalize os dados de entrada.
3. Implemente uma rede neural com:
   - 2 entradas (massa e temperatura)
   - 1 camada oculta com alguns neurônios (ex: 3 ou 4)
   -  saídas (representando os três tipos de reator)
4. Treine a rede com os dados fornecidos.
5. Teste com novos dados simulados, por exemplo:

<img src="img/Tab_09.png" alt="UESC" style="width: 350px; display: block; margin: 0 auto 5px auto;">
<p style="text-align: center;"><strong>Figura: Tabela de dados para previsão de novas correntes.</strong></p>

6. Analise a previsão da rede e discuta se a classificação parece coerente com os dados de entrada.

#### Implementação em Python

__Exercício – Classificação do Tipo de Reator com Redes Neurais__:

O objetivo é construir uma rede neural simples para classificar o tipo de reator com base na massa de catalisador e temperatura de operação. Usaremos uma abordagem passo a passo, com implementação do zero (sem Keras ou Scikit-learn), ideal para ensino.

1. __Dados brutos__

In [8]:
import numpy as np

# Dados de entrada: [massa (g), temperatura (°C)]
entradas = np.array([
    [250, 320],  # R-101
    [300, 350],  # R-102
    [275, 340],  # R-103
    [225, 310],  # R-104
], dtype=np.float32)

# Saída (one-hot encoding):
# CSTR = [1, 0, 0], PFR = [0, 1, 0], BATCH = [0, 0, 1]
saidas = np.array([
    [1, 0, 0],  # R-101 → CSTR
    [0, 1, 0],  # R-102 → PFR
    [1, 0, 0],  # R-103 → CSTR
    [0, 0, 1],  # R-104 → BATCH
], dtype=np.float32)


2. . Normalização das entradas

In [9]:
def normalizar(dados):
    minimo = dados.min(axis=0)
    maximo = dados.max(axis=0)
    return (dados - minimo) / (maximo - minimo), minimo, maximo

entradas_norm, entradas_min, entradas_max = normalizar(entradas)


3. Funções de ativação (ReLU e Softmax)

In [10]:
def relu(x):
    return np.maximum(0, x)

def relu_derivada(x):
    return (x > 0).astype(float)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # estabilidade numérica
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)


4. Inicialização da rede neural

In [11]:
np.random.seed(0)

# Estrutura: 2 entradas, 4 neurônios ocultos, 3 saídas
pesos_entrada_oculta = np.random.randn(2, 4)
bias_oculta = np.zeros((1, 4))

pesos_oculta_saida = np.random.randn(4, 3)
bias_saida = np.zeros((1, 3))

taxa_aprendizado = 0.1
epochs = 1000


5. Treinamento

In [12]:
for epoca in range(epochs):
    # Forward
    z_oculta = np.dot(entradas_norm, pesos_entrada_oculta) + bias_oculta
    a_oculta = relu(z_oculta)

    z_saida = np.dot(a_oculta, pesos_oculta_saida) + bias_saida
    saida_predita = softmax(z_saida)

    # Erro (categorical cross-entropy simplificada)
    erro = saidas - saida_predita

    # Backpropagation
    d_saida = erro  # derivada direta da cross-entropy com softmax

    d_pesos_oculta_saida = np.dot(a_oculta.T, d_saida)
    d_bias_saida = np.sum(d_saida, axis=0, keepdims=True)

    d_oculta = np.dot(d_saida, pesos_oculta_saida.T) * relu_derivada(z_oculta)
    d_pesos_entrada_oculta = np.dot(entradas_norm.T, d_oculta)
    d_bias_oculta = np.sum(d_oculta, axis=0, keepdims=True)

    # Atualização dos pesos e bias
    pesos_oculta_saida += taxa_aprendizado * d_pesos_oculta_saida
    bias_saida += taxa_aprendizado * d_bias_saida

    pesos_entrada_oculta += taxa_aprendizado * d_pesos_entrada_oculta
    bias_oculta += taxa_aprendizado * d_bias_oculta

    # Erro médio
    if epoca % 200 == 0:
        loss = np.mean(np.square(erro))
        print(f'Época {epoca} - Erro quadrático médio: {loss:.4f}')


Época 0 - Erro quadrático médio: 0.2530
Época 200 - Erro quadrático médio: 0.0058
Época 400 - Erro quadrático médio: 0.0005
Época 600 - Erro quadrático médio: 0.0001
Época 800 - Erro quadrático médio: 0.0001


6. Previsão com novos dados

In [13]:
# Novas entradas: [massa, temperatura]
novos = np.array([
    [260, 330],
    [290, 345]
], dtype=np.float32)

# Normalizar com base no conjunto original
novos_norm = (novos - entradas_min) / (entradas_max - entradas_min)

# Forward para novas previsões
z_oculta_novos = np.dot(novos_norm, pesos_entrada_oculta) + bias_oculta
a_oculta_novos = relu(z_oculta_novos)

z_saida_novos = np.dot(a_oculta_novos, pesos_oculta_saida) + bias_saida
saida_novos = softmax(z_saida_novos)

# Interpretar as previsões
tipos = ['CSTR', 'PFR', 'BATCH']
print("\nPrevisões para novos reatores:")
for i, pred in enumerate(saida_novos):
    tipo_previsto = tipos[np.argmax(pred)]
    print(f"Reator {i+1}: tipo previsto = {tipo_previsto} (distribuição: {pred.round(3)})")



Previsões para novos reatores:
Reator 1: tipo previsto = CSTR (distribuição: [0.997 0.003 0.   ])
Reator 2: tipo previsto = PFR (distribuição: [0.292 0.708 0.   ])


Saída esperada (exemplo)

In [15]:
# Saída esperada (exemplo):
# Época 0 - Erro quadrático médio: 0.4073
# Época 200 - Erro quadrático médio: 0.0289
# Época 400 - Erro quadrático médio: 0.0085
# ...
# Previsões para novos reatores:
# Reator 1: tipo previsto = CSTR (distribuição: [0.957 0.037 0.006])
# Reator 2: tipo previsto = PFR (distribuição: [0.119 0.851 0.030])


 #### Conclusão didática:

 A rede neural conseguiu classificar corretamente os tipos de reator com base em duas variáveis operacionais: massa de catalisador e temperatura. Mesmo com um conjunto pequeno de treinamento, o modelo generalizou bem para novos dados, demonstrando a aplicabilidade de redes neurais para problemas de classificação na Engenharia Química.

<div style="text-align: center; font-size: 12px; color: gray; margin-top: 40px;">
  Este notebook foi desenvolvido no âmbito do Grupo de Pesquisas em Modelagem Computacional da UESC.<br>
  Todos os direitos reservados © 2025
</div>